In [1]:
import pandas as pd
from pathlib import Path

In [2]:
data_path = Path().cwd().parent/'data'/'external'/'ufc-master.csv'

In [3]:
df = pd.read_csv(data_path)

In [4]:
pattern = r'_dif$'
df_dif = df.filter(regex=(pattern))

In [5]:
pattern = r'^B_'
df_b = df.filter(regex=(pattern))

In [6]:
pattern = r'^R_'
df_r = df.filter(regex=(pattern))

In [7]:
other_cols = df_dif.columns.append(df_b.columns).append(df_r.columns)

In [8]:
df_info = df.drop(other_cols, axis=1)

In [9]:
df2 = pd.concat([df_info, df_dif], axis=1)

In [11]:
df3 = pd.concat([
    df2,
    df_b[['B_fighter', 'B_odds']],
    df_r[['R_fighter', 'R_odds']]
],
    axis=1)

In [12]:
df3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4588 entries, 0 to 4587
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    4588 non-null   object 
 1   location                4588 non-null   object 
 2   country                 4588 non-null   object 
 3   Winner                  4588 non-null   object 
 4   title_bout              4588 non-null   bool   
 5   weight_class            4588 non-null   object 
 6   gender                  4588 non-null   object 
 7   no_of_rounds            4588 non-null   int64  
 8   empty_arena             4588 non-null   int64  
 9   constant_1              4588 non-null   int64  
 10  better_rank             4588 non-null   object 
 11  finish                  4118 non-null   object 
 12  finish_details          2014 non-null   object 
 13  finish_round            4099 non-null   float64
 14  finish_round_time       4099 non-null   

In [13]:
ufc = df3.drop(['finish_details', 'constant_1'], axis=1)

In [15]:
ufc.dropna(inplace=True)

In [19]:
# todo: run analysis and EDA for who will win predictions